In [1]:
from spacy.lang.en import English
import spacy

text = open('./lee-background.txt').read().lower()
nlp = spacy.load('en')
doc = nlp(text)

In [2]:
# apply spacy pipeline

docs = []
next_doc = []

for w in doc:
    if w.text != '\n' and not w.is_stop and not w.is_punct and not w.like_num:
        next_doc.append(w.lemma_)
    if w.text == '\n':
        docs.append(next_doc)
        next_doc = []

In [3]:
from gensim import corpora
from gensim.models.phrases import Phrases

bigram_model = Phrases(docs)

docs = [bigram_model[doc] for doc in docs]
#  twice for tri grams?
docs = [bigram_model[doc] for doc in docs]

/anaconda3/lib/python3.7/site-packages/gensim/models/phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [4]:
dictionary = corpora.Dictionary(docs)
corpus = [dictionary.doc2bow(doc) for doc in docs]

In [6]:
from gensim.models import LsiModel, LdaModel

print(LdaModel)

model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=10)



<class 'gensim.models.ldamodel.LdaModel'>


In [7]:
import pandas as pd

topics = model.get_topics()

tts = model.get_topic_terms(topicid=0, topn=5)

print(dictionary[10])

# df = pd.DataFrame()

print(tts)

burn
[(85, 0.023664504), (0, 0.01102052), (845, 0.007711251), (109, 0.006186229), (225, 0.005239921)]


In [8]:
from pyLDAvis.gensim import prepare

print(prepare)

ModuleNotFoundError: No module named 'pyLDAvis'